In [ ]:
# This is sample code for now I just made on the spot

# Importing libraries
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# Defining cosmological constraints
H0 = 70 #km/sec/Mpc 
Om = 0.3 #present-day matter density parameter
Ol = 0.7 #present-day dark energy density parameter 
Ok = 0 #assuming a flat curvature of space 


# Defining the Hubble Parameter function assuming flat Lambda-CDM model
def Hubble(z, Om, Ol, Ok, H0):
    return H0 * np.sqrt(Om * (1 + z)**3 + Ok * (1 + z)**2 + Ol)

# Generating an array of redshift values from 0 to 5
z_values = np.arange(0,5,0.1) 

# Calculate the Hubble parameter H(z) for the given array of redshift values
H = Hubble(z_values, Om, Ol, Ok, H0) 

# Establishing the labels for the plot
plt.plot(z_values,H)
plt.xlabel('z')
plt.ylabel('H(z) [km/s/Mpc]')
plt.title('Hubble Parameter H(z) vs. Redshift (z)')

# Saving and displaying the plot
plt.savefig('Hubble Plot Assignment 1.png', dpi=200, bbox_inches="tight")
plt.show()